# 03 - Train and Deploy

In [2]:
import boto3
import sagemaker
import time
from time import strftime

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sm_client = boto3.client("sagemaker")
region = boto_session.region_name
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]

prefix = 'sagemaker-intel2-dvc'

print(f"account: {account}")
print(f"bucket: {bucket}")
print(f"region: {region}")
print(f"role: {role}")

account: 991979990579
bucket: sagemaker-ap-south-1-991979990579
region: ap-south-1
role: arn:aws:iam::991979990579:role/service-role/AmazonSageMaker-ExecutionRole-20221229T112379


In [3]:
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import TensorBoardOutputConfig

Commit Hash: `bc94a47e4f16ee035785b14b92eb0bb43e479b20`

In [4]:
dvc_repo_url = "codecommit::ap-south-1://sagemaker-intel-classification2"
dvc_branch = "processed-dataset"

In [5]:
tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=f's3://{bucket}/sagemaker-intel-logs',
    container_local_output_path='/opt/ml/output/tensorboard'
)

In [6]:
!pwd

/root/Sagemaker-Pipeline/notebooks


In [19]:
pt_estimator = PyTorch(
    #image_uri="991979990579.dkr.ecr.ap-south-1.amazonaws.com/sagemaker:latest",
    base_job_name="training-intel-dataset",
    source_dir="/root/Sagemaker-Pipeline",
    entry_point="train.py",
    role=get_execution_role(),
    py_version="py38",
    framework_version="1.12.0",
    instance_count=1,
    # instance_type="ml.m5.large",
    #instance_type="ml.c5.4xlarge",
    instance_type="ml.g4dn.xlarge",
    tensorboard_output_config=tensorboard_output_config,
    use_spot_instances=True,
    max_wait=10000,
    max_run=10000,
    environment={
        "DVC_REPO_URL": dvc_repo_url,
        "DVC_BRANCH": dvc_branch,
        "GIT_USER": "Kandula Narasimharao",
        "GIT_EMAIL": "kandulanarasimharao@gmail.com"
    }
)

In [21]:
pt_estimator.fit()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: training-intel-dataset-2023-02-12-20-45-06-283


2023-02-12 20:45:07 Starting - Starting the training job...
2023-02-12 20:45:23 Starting - Preparing the instances for training.........
2023-02-12 20:46:58 Downloading - Downloading input data
2023-02-12 20:46:58 Training - Downloading the training image..................
2023-02-12 20:50:00 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-02-12 20:50:12,079 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-02-12 20:50:12,115 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-02-12 20:50:12,119 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-02-12 20:50:12,430 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requirements.txt
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━